<a href="https://colab.research.google.com/github/mariorizki-lang/mid-term-deep-learning/blob/main/midterm_regression_DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
# Cell 1: Instalasi dependensi dan pengunduhan dataset
!pip install -q gdown scikit-learn tensorflow

In [11]:
import pandas as pd

# Membaca file CSV, melewati baris yang rusak
raw_df = pd.read_csv('/content/midterm-regresi-dataset.csv', header=None, on_bad_lines='skip')

# Kolom pertama adalah target, sisanya fitur
raw_df.rename(columns={0: 'year'}, inplace=True)
feature_cols = [f'feature_{i}' for i in range(raw_df.shape[1] - 1)]
for idx, col in enumerate(feature_cols):
    raw_df.rename(columns={idx+1: col}, inplace=True)

raw_df.head()

/tmp/ipython-input-176965753.py:4: DtypeWarning: Columns (1,9,14,19,26,33,40) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_df = pd.read_csv('/content/midterm-regresi-dataset.csv', header=None, on_bad_lines='skip')


,year,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,feature_80,feature_81,feature_82,feature_83,feature_84,feature_85,feature_86,feature_87,feature_88,feature_89
0,2001.0,49.94357,21.47114,73.07750,8.74861,-17.40628,-13.09905,-25.01202,-12.23257,7.83089,...,13.01620,-54.40548,58.99367,15.37344,1.11144,-23.08793,68.40795,-1.82223,-27.46348,2.26327
1,2001.0,48.73215,18.42930,70.32679,12.94636,-10.32437,-24.83777,8.76630,-0.92019,18.76548,...,5.66812,-19.68073,33.04964,42.87836,-9.90378,-32.22788,70.49388,12.04941,58.43453,26.92061
2,2001.0,50.95714,31.85602,55.81851,13.41693,-6.57898,-18.54940,-3.27872,-2.35035,16.07017,...,3.03800,26.05866,-50.92779,10.93792,-0.07568,43.20130,-115.00698,-0.05859,39.67068,-0.66345
3,2001.0,48.2475,-1.89837,36.29772,2.58776,0.97170,-26.21683,5.05097,-10.34124,3.55005,...,34.57337,-171.70734,-16.96705,-46.67617,-12.51516,82.58061,-72.08993,9.90558,199.62971,18.85382
4,2001.0,50.9702,42.20998,67.09964,8.46791,-15.85279,-16.81409,-12.48207,-9.37636,12.63699,...,9.92661,-55.95724,64.92712,-17.72522,-1.49237,-7.50035,51.76631,7.88713,55.66926,28.74903


In [14]:
# Cell 3: Preprocessing – Imputasi nilai hilang dan scaling
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import numpy as np

# Pisahkan fitur dan target
X = raw_df[feature_cols]
y = raw_df['year']

# Convert all feature columns to numeric, coercing errors to NaN
for col in feature_cols:
    X[col] = pd.to_numeric(X[col], errors='coerce')

# Preprocessor: imputasi median dan standardisasi
preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline([
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())
        ]), feature_cols)
    ]
)

# Split data menjadi train dan test
# (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Terapkan preprocessing ke training dan test
X_train_prepared = preprocessor.fit_transform(X_train)
X_test_prepared = preprocessor.transform(X_test)

# Ubah target ke array numpy
y_train = y_train.values
y_test = y_test.values

print('Preprocessing selesai. Bentuk data latih:', X_train_prepared.shape)

/tmp/ipython-input-1970142883.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = pd.to_numeric(X[col], errors='coerce')


Preprocessing selesai. Bentuk data latih: (65311, 90)


In [15]:
# Cell 4: Membangun dan melatih model jaringan saraf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Definisikan arsitektur model
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_prepared.shape[1],)),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.1),
    Dense(1)  # Output layer (regresi)
])

# Kompilasi model
model.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae', 'mse']
)

# Callback untuk early stopping
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Melatih model
history = model.fit(
    X_train_prepared, y_train,
    validation_split=0.2,
    epochs=100,
    batch_size=64,
    callbacks=[early_stop],
    verbose=2
)


Epoch 1/100


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


817/817 - 4s - 5ms/step - loss: 1104569.3750 - mae: 822.5142 - mse: 1104569.3750 - val_loss: 216375.4375 - val_mae: 357.5061 - val_mse: 216375.4375
Epoch 2/100
817/817 - 3s - 4ms/step - loss: 180928.8750 - mae: 322.0316 - mse: 180928.8750 - val_loss: 101374.7422 - val_mae: 238.6938 - val_mse: 101374.7422
Epoch 3/100
817/817 - 4s - 5ms/step - loss: 102236.9375 - mae: 237.6485 - mse: 102236.9375 - val_loss: 49707.6250 - val_mae: 162.9087 - val_mse: 49707.6250
Epoch 4/100
817/817 - 2s - 3ms/step - loss: 62667.4609 - mae: 183.6890 - mse: 62667.4609 - val_loss: 22152.7324 - val_mae: 102.1827 - val_mse: 22152.7324
Epoch 5/100
817/817 - 2s - 3ms/step - loss: 43781.9531 - mae: 151.3797 - mse: 43781.9531 - val_loss: 11451.3330 - val_mae: 69.0418 - val_mse: 11451.3330
Epoch 6/100
817/817 - 2s - 3ms/step - loss: 35298.6211 - mae: 136.1911 - mse: 35298.6211 - val_loss: 6031.1792 - val_mae: 47.3569 - val_mse: 6031.1792
Epoch 7/100
817/817 - 3s - 4ms/step - loss: 31564.9355 - mae: 128.3713 - mse: 31

In [16]:
# Cell 5: Evaluasi model pada data uji
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Prediksi pada data test
y_pred = model.predict(X_test_prepared).flatten()

# Hitung metrik evaluasi
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print('Deep Learning Model Performance:')
print('MSE :', mse)
print('RMSE:', rmse)
print('MAE :', mae)
print('R²  :', r2)

511/511 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Deep Learning Model Performance:
MSE : 13840.64449767165
RMSE: 117.64626852421479
MAE : 15.893246797882185
R²  : -126.72133596182101
